In [1]:
import cv2
import numpy as np

"""
    Non-max Suppression Algorithm
    @param list  Object candidate bounding boxes
    @param list  Confidence score of bounding boxes
    @param float IoU threshold
    @return Rest boxes after nms operation
"""
def nms(bounding_boxes: list[tuple], 
        confidence_score: list[float], 
        threshold: float) -> tuple[list[tuple], list[float]]:
    # If no bounding boxes, return empty list
    if len(bounding_boxes) == 0:
        return [], []

    # Bounding boxes
    boxes = np.array(bounding_boxes)

    # coordinates of bounding boxes
    start_x = boxes[:, 0]
    start_y = boxes[:, 1]
    end_x = boxes[:, 2]
    end_y = boxes[:, 3]

    # Confidence scores of bounding boxes
    score = np.array(confidence_score)

    # Picked bounding boxes
    picked_boxes = []
    picked_score = []

    # Compute areas of bounding boxes
    areas = (end_x - start_x + 1) * (end_y - start_y + 1)

    # Sort by confidence score of bounding boxes
    order = np.argsort(score)

    # Iterate bounding boxes
    while order.size > 0:
        # The index of largest confidence score
        index = order[-1]

        # Pick the bounding box with largest confidence score
        picked_boxes.append(bounding_boxes[index])
        picked_score.append(confidence_score[index])

        # Compute ordinates of intersection-over-union(IOU)
        remain = order[:-1]
        x1 = np.maximum(start_x[index], start_x[remain])
        x2 = np.minimum(end_x[index], end_x[remain])
        y1 = np.maximum(start_y[index], start_y[remain])
        y2 = np.minimum(end_y[index], end_y[remain])

        # Compute areas of intersection-over-union
        w = np.maximum(0.0, x2 - x1 + 1)
        h = np.maximum(0.0, y2 - y1 + 1)
        intersection = w * h

        # Compute the ratio between intersection and union
        ratio = intersection / (areas[index] + areas[remain] - intersection)

        left = np.where(ratio < threshold)
        order = order[left]

    return picked_boxes, picked_score

In [2]:
def myNMS(bounding_boxes: list[tuple], confidence_score: list[tuple], threshold: float) -> tuple[list[tuple], list[float]]:
    
    if (len(bounding_boxes) == 0):
        return [], []
    
    boxes = np.array(bounding_boxes)
    scores = np.array(confidence_score)
    
    start_x = boxes[:, 0]
    start_y = boxes[:, 1]
    end_x = boxes[:, 2]
    end_y = boxes[:, 3]
    
    # compute the area of each bounding box
    areas = (end_x - start_x + 1) * (end_y - start_y + 1)
    
    picked_boxes = []
    picked_scores = []
    
    # sort all the scores in increasing order
    order = np.argsort(scores)
    
    while order.size > 0:
        # highest index
        index = order[-1]
        picked_boxes.append(bounding_boxes[index])
        picked_scores.append(confidence_score[index])
        
        # compute IOU between boxes[index] and others, np broadcasting is used
        x1 = np.maximum(start_x[index], start_x[order[:-1]])
        y1 = np.maximum(start_y[index], start_y[order[:-1]])
        x2 = np.minimum(end_x[index], end_x[order[:-1]])
        y2 = np.minimum(end_y[index], end_y[order[:-1]])
        
        w = np.maximum(x2 - x1, 0.0)
        h = np.maximum(y2 - y1, 0.0)
        intersection = w * h
        
        iou_ratio = intersection / (areas[index] + areas[order[:-1]] - intersection)
        
        # keep the boxes that has a IOU smaller than the threshold
        left = np.where(iou_ratio < threshold)
        order = order[left]
        
    return picked_boxes, picked_scores

In [4]:
def MyNMS2(bounding_boxes, confidence_score, threshold):
    if (len(bounding_boxes) == 0):
        return [], []
    
    boxes = np.array(bounding_boxes)
    scores = np.array(confidence_score)
    threshold = threshold
    
    start_x = boxes[:, 0]
    start_y = boxes[:, 1]
    end_x = boxes[:, 2]
    end_y = boxes[:, 3]
    
    # compute the areas
    areas = (end_x - start_x + 1) * (end_y - start_y + 1)
    
    picked_boxes = []
    picked_scores = []
    
    # sort based on confidence score
    order = np.argsort(scores)
    
    while order.size > 0:
        # index with highest score
        index = order[-1]
        picked_boxes.append(bounding_boxes[index])
        picked_scores.append(confidence_score[index])
        
        x1 = np.maximum(start_x[index], start_x[:-1])
        y1 = np.maximum(start_y[index], start_y[:-1])
        x2 = np.minimum(end_x[index], end_x[:-1])
        y2 = np.minimum(end_y[index], end_y[:-1])
        
        w = np.maximum(x2 - x1, 0)
        h = np.maximum(y2 - y1, 0)
        intersection = w * h
        
        iou = intersection / (areas[index] + areas[order[:-1]] - intersection)
        
        # keep the boxes that has a IOU smaller than the threshold
        left = np.where(iou < threshold)
        order = order[left]
        
    return picked_boxes, picked_scores

In [ ]:
# Image name
image_name = 'nms.jpg'

# Bounding boxes
bounding_boxes = [(187, 82, 337, 317), (150, 67, 305, 282), (246, 121, 368, 304)]
confidence_score = [0.9, 0.75, 0.8]

# Read image
image = cv2.imread(image_name)

# Copy image as original
org = image.copy()

# Draw parameters
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
thickness = 2

# IoU threshold
threshold = 0.4

# Draw bounding boxes and confidence score
for (start_x, start_y, end_x, end_y), confidence in zip(bounding_boxes, confidence_score):
    (w, h), baseline = cv2.getTextSize(str(confidence), font, font_scale, thickness)
    cv2.rectangle(org, (start_x, start_y - (2 * baseline + 5)), (start_x + w, start_y), (0, 255, 255), -1)
    cv2.rectangle(org, (start_x, start_y), (end_x, end_y), (0, 255, 255), 2)
    cv2.putText(org, str(confidence), (start_x, start_y), font, font_scale, (0, 0, 0), thickness)

# Run non-max suppression algorithm
picked_boxes, picked_score = MyNMS2(bounding_boxes, confidence_score, threshold)

# Draw bounding boxes and confidence score after non-maximum supression
for (start_x, start_y, end_x, end_y), confidence in zip(picked_boxes, picked_score):
    (w, h), baseline = cv2.getTextSize(str(confidence), font, font_scale, thickness)
    cv2.rectangle(image, (start_x, start_y - (2 * baseline + 5)), (start_x + w, start_y), (0, 255, 255), -1)
    cv2.rectangle(image, (start_x, start_y), (end_x, end_y), (0, 255, 255), 2)
    cv2.putText(image, str(confidence), (start_x, start_y), font, font_scale, (0, 0, 0), thickness)

# Show image
cv2.imshow('Original', org)
cv2.imshow('NMS', image)
cv2.waitKey(0)